In [1]:
! pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# IMPORTAÇÃO BIBLIOTECAS
import pymysql
import pandas as pd
import warnings
from IPython.display import display, Markdown

In [3]:
def exibir_markdown(texto_md):
    display(Markdown(texto_md))

In [4]:
# DESATIVAR ALERTAS
warnings.filterwarnings('ignore')

In [5]:
# CRIAR CONEXÃO AO MYSQL - AWS

try:
  conexao = pymysql.connect( user= 'clara_ferreira',
                             host= 'ada-santander-1007.clzgkgrmzmzh.us-east-1.rds.amazonaws.com',
                             port= 3306,
                             password= '123456',
                             database= 'clara_ferreira_teste')
except pymysql.Error as e:
  print('Erro ao se conectar: ', e)

In [6]:
# CRIAR O CURSOR

cursor = conexao.cursor()

In [7]:
#DELETANDO TABELAS
drop_exists_produto = 'DROP TABLE IF EXISTS produto'
drop_exists_categoria = 'DROP TABLE IF EXISTS categoria'

cursor.execute(drop_exists_produto)
cursor.execute(drop_exists_categoria)


0

In [8]:
# CATEGORIA (ID_CATEGORIA, CATEGORIA)
# PRODUTO (ID_PRODUTO, ID_CATEGORIA, PRODUTO)
ddl_categoria = """ CREATE TABLE categoria (
                  id_categoria int auto_increment primary key,
                  categoria varchar(200)
                 ) """

In [9]:
# EXECUÇÃO DE COMANDO SQL
cursor.execute(ddl_categoria)

0

In [10]:
# COMANDO DDL PRODUTO

ddl_produto = """  CREATE TABLE produto (
                id_produto int auto_increment primary key,
                id_categoria int,
                produto varchar(200) not null,
                foreign key (id_categoria) references categoria (id_categoria)
              ) """


In [11]:
# EXECUÇÃO DE COMANDO SQL
cursor.execute(ddl_produto)

0

In [12]:
# INSERIR REGISTROS DE CATEGORIA
dml_insert_categoria = """ INSERT INTO categoria (categoria) VALUES
                            ('Limpeza'),
                            ('Fruta')
"""

In [13]:
# EXECUTAR COMANDO DML
cursor.execute(dml_insert_categoria)

2

In [14]:
#COMMIT
conexao.commit()

In [15]:
# INSERIR REGISTROS DE PRODUTO
dml_insert_produto = """ INSERT INTO produto (id_categoria, produto) VALUES
                          (1, 'Sabão em pó'),
                          (1, 'Papel higiênico'),
                          (null, 'Sorvete')

"""

In [16]:
# EXECUTAR COMANDO SQL
cursor.execute(dml_insert_produto)

3

In [17]:
# COMMIT
conexao.commit()

In [18]:
## CRIAR CURSOR

cursor = conexao.cursor()

In [19]:
# SOLICITAR O NOVO CADASTRO
# INSERIR O NOVO CADASTRO NO BD
# MOSTRAR O REGISTRO INSERIDO
def inserir_categoria():
  nova_categoria = input('Digite o nome da CATEGORIA que desja cadastrar: ')

  sql_insert_categoria = "insert into categoria (categoria) values ('{}')".format(nova_categoria)

  cursor.execute(sql_insert_categoria)
  conexao.commit()

  print('\n ---------- CADASTRO EFETUADO COM SUCESSO! ----------')

  df_categoria = pd.read_sql_query('select * from categoria', conexao)
  print(df_categoria.to_markdown(index=False))


In [20]:
# ID_PRODUTO, ID_CATEGORIA, PRODUTO
# SOLICITAR O NOME DO PRODUTO
# SOLICITAR A CATEGORIA
# CADASTRAR O PRODUTO
# EXIBE O PRODUTO CADASTRADO

def inserir_produto():
  novo_produto = input('Qual PRODUTO deseja cadastrar: ')

  df_categoria = pd.read_sql_query('select * from categoria', conexao)
  print(df_categoria.to_markdown(index=False))

  id_categoria = input('O produto '+novo_produto+' pertence a qual categoria? Use 0 para nenhuma: ')

  # verificar de id_categoria = 0
  if int(id_categoria) == 0:
    sql_insert_produto = f"insert into produto (produto) values ('{novo_produto}')"
  else:
    sql_insert_produto = f"insert into produto (id_categoria, produto) values({int(id_categoria)}, '{novo_produto}')"

  cursor.execute(sql_insert_produto)
  conexao.commit()

  print('\n ----- CADASTRO EFETUADO COM SUCESSO -----')

  df_produto = pd.read_sql_query(""" select id_produto, categoria, produto
                                     from produto p
                                     left join categoria c on c.id_categoria = p.id_categoria """, conexao)
  exibir_markdown(df_produto.to_markdown(index=False))




In [21]:
# ALTERAR UMA CATEGORIA
# SELECIONAR A CATEGORIA QUE SERÁ EDITADA
# RECEBER O NOVO VALOR DA CATEGORIA
# EDITAR A CATEGORIA

def alterar_categoria():
  df_categoria = pd.read_sql_query('select * from categoria', conexao)
  exibir_markdown(df_categoria.to_markdown(index=False))

  id_categoria = input('Digite o ID da categoria a ser editada: ')
  valor_novo_categoria = input('Novo valor da Categoria: ')

  sql_alterar_categoria = f""" update categoria
                               set categoria = '{valor_novo_categoria}'
                               where id_categoria = {int(id_categoria)} """

  cursor.execute(sql_alterar_categoria)
  conexao.commit()

  df_categoria = pd.read_sql_query('select * from categoria', conexao)
  exibir_markdown(df_categoria.to_markdown(index=False))




In [22]:
# ALTERAR PRODUTO
# SELECIONAR PRODUTO A SER EDITADO
# SELECIONAR A CATEGORIA REFERENTE AO PRODUTO
# INSERIR ALTERAÇÃO COM NOVOS VALORES

def alterar_produto():
  df_produto = pd.read_sql_query(""" select id_produto, categoria, produto
                                     from produto p
                                     left join categoria c on c.id_categoria = p.id_categoria """, conexao)
  exibir_markdown(df_produto.to_markdown(index=False))

  # selecionando o produto a ser editado e recebendo  novo valor da descrição do produto
  id_produto = input('Digite o ID do produto a ser editado: ')
  valor_novo_produto = input('Qual o novo valor para DESCRIÇÃO DO PRODUTO: ')

  # selecionando a categoria que o produto pertence
  df_categoria = pd.read_sql_query('select * from categoria', conexao)
  exibir_markdown(df_categoria.to_markdown(index=False))

  valor_novo_id_categoria = input('Digite o ID da Categoria do produto '+valor_novo_produto+' (use 0 para nenhuma): ')

  if int(valor_novo_id_categoria) == 0:
    sql_alterar_produto = f""" update produto
                               set id_categoria = null, produto = '{valor_novo_produto}'
                               where id_produto = {int(id_produto)} """
  else:
    sql_alterar_produto = f""" update produto
                               set id_categoria = {int(valor_novo_id_categoria)},
                                    produto = '{valor_novo_produto}'
                              where id_produto = {int(id_produto)} """


  cursor.execute(sql_alterar_produto)
  conexao.commit()


  df_produto = pd.read_sql_query(""" select id_produto, categoria, produto
                                     from produto p
                                     left join categoria c on c.id_categoria = p.id_categoria """, conexao)
  exibir_markdown(df_produto.to_markdown(index=False))







In [23]:
### INSERIR / ALTERAR / RECUPERAR DADOS DO MYSQL ATRAVÉS DO PYTHON

cursor = conexao.cursor()

while True:

  opcao_menu = input("""
  Selecione a opção desejada:
  1- Cadastrar Categoria
  2- Cadastrar Produto
  3- Alterar Categoria
  4- Alterar Produto
  5- Sair
  """)

  if opcao_menu == '1':
    inserir_categoria()
  elif opcao_menu == '2':
    inserir_produto()
  elif opcao_menu == '3':
    alterar_categoria()
  elif opcao_menu == '4':
    alterar_produto()
  elif opcao_menu == '5':
    print('ENCERRAR PROGRAMA')
    cursor.close()
    break
  else:
    print('OPÇÃO INVÁLIDA')


ENCERRAR PROGRAMA
